<a href="https://colab.research.google.com/github/Joseph-Moon/hello-world/blob/master/rag1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip3 install -qU  markdownify  langchain-upstage rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.8/355.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.4/328.4 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%pip install --quiet openai python-dotenv

In [ ]:
import dotenv
dotenv.load_dotenv('/content/drive/MyDrive/Colab Notebooks/.env')

True

In [ ]:
from langchain_upstage import UpstageLayoutAnalysisLoader

layzer = UpstageLayoutAnalysisLoader(
    "/content/drive/MyDrive/Colab Notebooks/GenAI_Handbook.pdf", use_ocr=True, output_type="html"
)
# For improved memory efficiency, consider using the lazy_load method to load documents page by page.
docs = layzer.load()  # or layzer.lazy_load()

In [ ]:
from IPython.display import display, HTML

display(HTML(docs[0].page_content[:1000]))

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer from the following context.
    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {question}
    ---
    Context: {Context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [ ]:
%pip install -U --quiet langchain_community langchain_text_splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.5 MB/s eta 0:00:00


In [ ]:
from langchain_community.retrievers import BM25Retriever
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=1000, chunk_overlap=100, language=Language.HTML
)
splits = text_splitter.split_documents(docs)

retriever = BM25Retriever.from_documents(splits)

In [ ]:
query = "what do developers do to reduce the number of LLM requests?"
context_docs = retriever.invoke(query)
chain.invoke({"question": query, "Context": context_docs})

'Developers can reduce the number of LLM requests by batching requests or implementing delays between subsequent requests. Additionally, they should implement a process to track the number of requests sent or identify this from the error message/API endpoint to determine the degree to which they need to limit the requests.'

In [ ]:
query = "what is Parent-Child Document Retriever?"
context_docs = retriever.invoke(query)
chain.invoke({"question": query, "Context": context_docs})

'The Parent-Child Document Retriever is a component of the Retriever in RAG (Retrieve and Generate) applications that utilizes a multi-level chunking strategy. It addresses the challenge of determining the optimal chunk size for vector similarity search and LLM prompt inclusion. The Parent-Child Document Retriever creates two chunk sizes and two chunk overlaps: one for large chunks and one for small chunks. The original data is first split into large chunks, and then the large chunks are split into small chunks. The small chunks contain a reference to the large chunks they were derived from. The small chunks are used to create vector embeddings, which are used during similarity search. This component of the retriever (child) enables the application to realize the benefits of smaller chunks, which contain less noise. After the application retrieves the most similar small chunks, it obtains the corresponding large chunks, and includes these chunks in the LLM prompt. This component of the

In [ ]:
query = "Why do the RAG applications benefit from using a Parent-Child Document Retriever?"
context_docs = retriever.invoke(query)
chain.invoke({"question": query, "Context": context_docs})

'The information is not present in the context.'